# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Lincoln,-34.7333,135.8667,57.49,67,100,16.62,AU,1658883297
1,1,Bajil,15.0584,43.2851,87.46,49,68,4.56,YE,1658883299
2,2,Dikson,73.5069,80.5464,43.52,97,100,22.48,RU,1658883301
3,3,Barcelos,41.5388,-8.6151,65.84,87,0,2.53,PT,1658883304
4,4,Butaritari,3.0707,172.7902,81.90,76,6,19.69,KI,1658883171


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Create the Google Maps figure
fig = gmaps.figure()

# Convert humidity column to a list
humidity = weather_data["Humidity"].astype(float)

# Create marker locations using the city coordinates
marker_locations = weather_data[['Lat', 'Lng']]

# Create a marker_layer using the marker locations
#markers = gmaps.marker_layer(marker_locations)

# Create a heat_layer using the humidity_list
heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

# Add the marker and heat layers
#fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
# Create a cleaned weather data df by copying the weather data df
cleaned_weather_data = weather_data.copy()

# Drop any rows with null values
cleaned_weather_data = weather_data.dropna()

# Filter the data to only include cities with max temperatures below 80 degrees
cleaned_weather_data = cleaned_weather_data.loc[cleaned_weather_data['Max Temp']<80, :]

# ... And above 70 degrees
cleaned_weather_data = cleaned_weather_data.loc[cleaned_weather_data['Max Temp']>70, :]

# Filter the data to only include cities with wind speeds less than 10 mph
cleaned_weather_data = cleaned_weather_data.loc[cleaned_weather_data['Wind Speed']<10, :]

# Filter the data to only include cities with zero cloudiness
cleaned_weather_data = cleaned_weather_data.loc[cleaned_weather_data['Cloudiness']==0, :]

# Display the cleaned city data
cleaned_weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Ponta do Sol,32.6667,-17.1000,74.12,60,0,5.86,PT,1658883324
18,18,São João da Barra,-21.6403,-41.0511,70.16,85,0,9.69,BR,1658883337
111,111,Jinchang,38.4953,102.1739,75.96,28,0,9.62,CN,1658883551
145,145,Mahon,39.8885,4.2658,73.96,73,0,3.44,ES,1658883633
171,171,Montenegro,42.5000,19.3000,73.15,65,0,2.30,ME,1658883696
248,248,Kiryat Gat,31.6100,34.7642,76.41,62,0,3.06,IL,1658883876
261,261,Palaiokomi,40.8667,23.9000,79.68,59,0,1.66,GR,1658883905
300,300,Bani Walid,31.7566,13.9942,72.90,52,0,3.27,LY,1658883996
319,319,Nantucket,41.2835,-70.0995,73.51,86,0,8.05,US,1658884040
453,453,Nouadhibou,20.9310,-17.0347,71.04,92,0,8.01,MR,1658884349


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Reset the index for cleaned_weather_data
hotel_df = cleaned_weather_data.copy()
hotel_df = hotel_df.reset_index()

# Delete the original index column
del hotel_df["index"]

# Create a list to store potential hotels
hotels = []

# Loop through the reset list of cities to find the nearest hotel
# Range function creates a series of indices to locate each city by row number
for city_index in range(len(hotel_df)):
    
    # Specify the latitude and longitude of the city
    lat = hotel_df.loc[city_index]["Lat"]
    lng = hotel_df.loc[city_index]["Lng"]
    
    # Arrange the latitude and longitude into a coordinate pair
    city_coords = f"{lat},{lng}"
    
    # Specify the search parameters
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    
    # Specify the base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    
    # Make request from Google API based on the search parameters
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()
    
    # Try to add the nearest hotel
    try:
        hotels.append(hotel_response["results"][0]["name"])
        
    # If a nearby hotel cannot be added, print "Nearest hotel not found"
    except:
        hotels.append("Nearest hotel not found")

# Add a column for the nearest hotel to the reset weather dataframe
hotel_df["Hotel Name"] = hotels
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,12,Ponta do Sol,32.6667,-17.1000,74.12,60,0,5.86,PT,1658883324,Hotel do Campo
1,18,São João da Barra,-21.6403,-41.0511,70.16,85,0,9.69,BR,1658883337,Pousada Porto De Canoas
2,111,Jinchang,38.4953,102.1739,75.96,28,0,9.62,CN,1658883551,Mingshihui Jinchang International Hotel
3,145,Mahon,39.8885,4.2658,73.96,73,0,3.44,ES,1658883633,ARTIEM Capri
4,171,Montenegro,42.5000,19.3000,73.15,65,0,2.30,ME,1658883696,Hostel IZVOR
5,248,Kiryat Gat,31.6100,34.7642,76.41,62,0,3.06,IL,1658883876,OlusHome
6,261,Palaiokomi,40.8667,23.9000,79.68,59,0,1.66,GR,1658883905,www.anastasia-villa.gr
7,300,Bani Walid,31.7566,13.9942,72.90,52,0,3.27,LY,1658883996,منزل مفتاح محمد الزلبي
8,319,Nantucket,41.2835,-70.0995,73.51,86,0,8.05,US,1658884040,Jared Coffin House
9,453,Nouadhibou,20.9310,-17.0347,71.04,92,0,8.01,MR,1658884349,El Medina


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))